In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [2]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [3]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohefinemols.npz")
data = data["arr_0"]

intdata = np.load("intfinemols.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([222262, 1, 53])
Integer dataset size: torch.Size([222262])


In [4]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((seq_length * i)) : int((seq_length * i) + (seq_length * batch_size)), :, :]
            targets = intdata[int((seq_length * i)) : int((seq_length * i) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                    
            return inp, target
        
        except:
            continue
        break

In [5]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
         
        #Initialize forget gate bias to 1    
        for names in self.lstm._all_weights:
            for name in filter(lambda n: "bias" in n, names):
                bias = getattr(self.lstm, name)
                n = bias.size(0)
                start, end = n//4, n//2
                
                nn.init.constant_(bias.data[start:end], 1.0)
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        return output, hidden

In [6]:
#Define training
def train():
    #Set start time
    start_time = time.time()
        
    #Iterate set of seq_length characters
    for i in range(int((np.shape(data)[0] - seq_length + 1) / batch_size)):
            
        #Initialize hidden and cell states
        hidden = model.init_states(num_layers, hidden_size)
            
        #Run on GPU if available
        if cuda:
            hidden = (hidden[0].cuda(), hidden[1].cuda())
        
        #Set initial gradients
        model.zero_grad()
    
        #Set initial loss
        loss = 0 
            
        #Get input and target
        input_data, target_data = inptarg(i, batch_size)
        input_data = input_data.float()
        target_data = target_data.long()
            
        #Run on GPU if available
        if cuda:
            input_data = input_data.cuda()
            target_data = target_data.cuda()
                
        #Run model, calculate loss
        output, hidden = model(input_data, hidden)
        loss += criterion(output.squeeze(), target_data.squeeze())
                
        #Backpropagate loss
        loss.backward()
        
        #Clip gradients
        nn.utils.clip_grad_norm_(model.parameters(), 3.0)
        
        #Optimize
        optimizer.step()
            
        #Update list of losses
        if (i % 50 == 0):
            losses[0] = loss.data.item() / seq_length   
        losses.append(loss.data.item() / seq_length)
            
        #Intermediary saves
        if (i % 250 == 0):
            torch.save(model.state_dict(), "network1.pth")

        #Print training info
        hours, minutes, seconds = time_elapsed(start_time)
        print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [7]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
seq_length = 75
batch_size = 128

#List of losses
losses = [0]

In [8]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
model.load_state_dict(torch.load("network.pth"))
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()

In [9]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 21267509


In [10]:
#Train
train()

Loss: 0.006888 | ΔLossTotal: +0.0000 | Iteration: 0001 | Time elapsed: 00h 00 m 04 s
Loss: 0.005338 | ΔLossTotal: -0.0016 | Iteration: 0002 | Time elapsed: 00h 00 m 08 s
Loss: 0.004819 | ΔLossTotal: -0.0021 | Iteration: 0003 | Time elapsed: 00h 00 m 13 s
Loss: 0.004659 | ΔLossTotal: -0.0022 | Iteration: 0004 | Time elapsed: 00h 00 m 17 s
Loss: 0.004338 | ΔLossTotal: -0.0026 | Iteration: 0005 | Time elapsed: 00h 00 m 21 s
Loss: 0.004185 | ΔLossTotal: -0.0027 | Iteration: 0006 | Time elapsed: 00h 00 m 25 s
Loss: 0.004010 | ΔLossTotal: -0.0029 | Iteration: 0007 | Time elapsed: 00h 00 m 29 s
Loss: 0.003883 | ΔLossTotal: -0.0030 | Iteration: 0008 | Time elapsed: 00h 00 m 33 s
Loss: 0.003788 | ΔLossTotal: -0.0031 | Iteration: 0009 | Time elapsed: 00h 00 m 37 s
Loss: 0.003641 | ΔLossTotal: -0.0032 | Iteration: 0010 | Time elapsed: 00h 00 m 42 s
Loss: 0.003487 | ΔLossTotal: -0.0034 | Iteration: 0011 | Time elapsed: 00h 00 m 46 s
Loss: 0.003459 | ΔLossTotal: -0.0034 | Iteration: 0012 | Time ela

Loss: 0.000898 | ΔLossTotal: -0.0060 | Iteration: 0098 | Time elapsed: 00h 06 m 59 s
Loss: 0.000832 | ΔLossTotal: -0.0061 | Iteration: 0099 | Time elapsed: 00h 07 m 03 s
Loss: 0.000888 | ΔLossTotal: -0.0060 | Iteration: 0100 | Time elapsed: 00h 07 m 08 s
Loss: 0.000831 | ΔLossTotal: -0.0061 | Iteration: 0101 | Time elapsed: 00h 07 m 12 s
Loss: 0.000862 | ΔLossTotal: -0.0060 | Iteration: 0102 | Time elapsed: 00h 07 m 16 s
Loss: 0.000895 | ΔLossTotal: -0.0060 | Iteration: 0103 | Time elapsed: 00h 07 m 21 s
Loss: 0.000974 | ΔLossTotal: -0.0059 | Iteration: 0104 | Time elapsed: 00h 07 m 25 s
Loss: 0.000991 | ΔLossTotal: -0.0059 | Iteration: 0105 | Time elapsed: 00h 07 m 29 s
Loss: 0.001026 | ΔLossTotal: -0.0059 | Iteration: 0106 | Time elapsed: 00h 07 m 34 s
Loss: 0.001038 | ΔLossTotal: -0.0059 | Iteration: 0107 | Time elapsed: 00h 07 m 38 s
Loss: 0.001045 | ΔLossTotal: -0.0058 | Iteration: 0108 | Time elapsed: 00h 07 m 42 s
Loss: 0.001050 | ΔLossTotal: -0.0058 | Iteration: 0109 | Time ela

Loss: 0.000874 | ΔLossTotal: -0.0060 | Iteration: 0195 | Time elapsed: 00h 13 m 41 s
Loss: 0.000885 | ΔLossTotal: -0.0060 | Iteration: 0196 | Time elapsed: 00h 13 m 45 s
Loss: 0.000977 | ΔLossTotal: -0.0059 | Iteration: 0197 | Time elapsed: 00h 13 m 49 s
Loss: 0.000939 | ΔLossTotal: -0.0059 | Iteration: 0198 | Time elapsed: 00h 13 m 53 s
Loss: 0.000968 | ΔLossTotal: -0.0059 | Iteration: 0199 | Time elapsed: 00h 13 m 57 s
Loss: 0.001028 | ΔLossTotal: -0.0059 | Iteration: 0200 | Time elapsed: 00h 14 m 01 s
Loss: 0.000984 | ΔLossTotal: -0.0059 | Iteration: 0201 | Time elapsed: 00h 14 m 05 s
Loss: 0.001024 | ΔLossTotal: -0.0059 | Iteration: 0202 | Time elapsed: 00h 14 m 10 s
Loss: 0.001030 | ΔLossTotal: -0.0059 | Iteration: 0203 | Time elapsed: 00h 14 m 14 s
Loss: 0.001011 | ΔLossTotal: -0.0059 | Iteration: 0204 | Time elapsed: 00h 14 m 18 s
Loss: 0.000987 | ΔLossTotal: -0.0059 | Iteration: 0205 | Time elapsed: 00h 14 m 22 s
Loss: 0.001026 | ΔLossTotal: -0.0059 | Iteration: 0206 | Time ela

Loss: 0.000944 | ΔLossTotal: -0.0059 | Iteration: 0292 | Time elapsed: 00h 20 m 16 s
Loss: 0.000922 | ΔLossTotal: -0.0060 | Iteration: 0293 | Time elapsed: 00h 20 m 20 s
Loss: 0.001000 | ΔLossTotal: -0.0059 | Iteration: 0294 | Time elapsed: 00h 20 m 24 s
Loss: 0.000965 | ΔLossTotal: -0.0059 | Iteration: 0295 | Time elapsed: 00h 20 m 28 s
Loss: 0.000935 | ΔLossTotal: -0.0060 | Iteration: 0296 | Time elapsed: 00h 20 m 32 s
Loss: 0.000945 | ΔLossTotal: -0.0059 | Iteration: 0297 | Time elapsed: 00h 20 m 36 s
Loss: 0.000903 | ΔLossTotal: -0.0060 | Iteration: 0298 | Time elapsed: 00h 20 m 40 s
Loss: 0.000860 | ΔLossTotal: -0.0060 | Iteration: 0299 | Time elapsed: 00h 20 m 44 s
Loss: 0.000884 | ΔLossTotal: -0.0060 | Iteration: 0300 | Time elapsed: 00h 20 m 48 s
Loss: 0.000915 | ΔLossTotal: -0.0060 | Iteration: 0301 | Time elapsed: 00h 20 m 52 s
Loss: 0.000981 | ΔLossTotal: -0.0059 | Iteration: 0302 | Time elapsed: 00h 20 m 56 s
Loss: 0.000963 | ΔLossTotal: -0.0059 | Iteration: 0303 | Time ela

Loss: 0.001012 | ΔLossTotal: -0.0059 | Iteration: 0389 | Time elapsed: 00h 26 m 50 s
Loss: 0.001094 | ΔLossTotal: -0.0058 | Iteration: 0390 | Time elapsed: 00h 26 m 54 s
Loss: 0.001090 | ΔLossTotal: -0.0058 | Iteration: 0391 | Time elapsed: 00h 26 m 58 s
Loss: 0.000996 | ΔLossTotal: -0.0059 | Iteration: 0392 | Time elapsed: 00h 27 m 02 s
Loss: 0.001028 | ΔLossTotal: -0.0059 | Iteration: 0393 | Time elapsed: 00h 27 m 06 s
Loss: 0.000972 | ΔLossTotal: -0.0059 | Iteration: 0394 | Time elapsed: 00h 27 m 11 s
Loss: 0.001004 | ΔLossTotal: -0.0059 | Iteration: 0395 | Time elapsed: 00h 27 m 15 s
Loss: 0.001007 | ΔLossTotal: -0.0059 | Iteration: 0396 | Time elapsed: 00h 27 m 19 s
Loss: 0.000989 | ΔLossTotal: -0.0059 | Iteration: 0397 | Time elapsed: 00h 27 m 23 s
Loss: 0.001051 | ΔLossTotal: -0.0058 | Iteration: 0398 | Time elapsed: 00h 27 m 27 s
Loss: 0.001089 | ΔLossTotal: -0.0058 | Iteration: 0399 | Time elapsed: 00h 27 m 31 s
Loss: 0.000942 | ΔLossTotal: -0.0059 | Iteration: 0400 | Time ela

...

Loss: 0.000825 | ΔLossTotal: -0.0061 | Iteration: 1262 | Time elapsed: 01h 26 m 02 s
Loss: 0.000783 | ΔLossTotal: -0.0061 | Iteration: 1263 | Time elapsed: 01h 26 m 06 s
Loss: 0.000848 | ΔLossTotal: -0.0060 | Iteration: 1264 | Time elapsed: 01h 26 m 10 s
Loss: 0.000864 | ΔLossTotal: -0.0060 | Iteration: 1265 | Time elapsed: 01h 26 m 14 s
Loss: 0.000913 | ΔLossTotal: -0.0060 | Iteration: 1266 | Time elapsed: 01h 26 m 18 s
Loss: 0.000873 | ΔLossTotal: -0.0060 | Iteration: 1267 | Time elapsed: 01h 26 m 23 s
Loss: 0.000833 | ΔLossTotal: -0.0061 | Iteration: 1268 | Time elapsed: 01h 26 m 27 s
Loss: 0.000884 | ΔLossTotal: -0.0060 | Iteration: 1269 | Time elapsed: 01h 26 m 31 s
Loss: 0.000847 | ΔLossTotal: -0.0060 | Iteration: 1270 | Time elapsed: 01h 26 m 35 s
Loss: 0.000934 | ΔLossTotal: -0.0060 | Iteration: 1271 | Time elapsed: 01h 26 m 39 s
Loss: 0.000907 | ΔLossTotal: -0.0060 | Iteration: 1272 | Time elapsed: 01h 26 m 43 s
Loss: 0.000922 | ΔLossTotal: -0.0060 | Iteration: 1273 | Time ela

Loss: 0.000969 | ΔLossTotal: -0.0059 | Iteration: 1359 | Time elapsed: 01h 32 m 36 s
Loss: 0.000933 | ΔLossTotal: -0.0060 | Iteration: 1360 | Time elapsed: 01h 32 m 41 s
Loss: 0.000978 | ΔLossTotal: -0.0059 | Iteration: 1361 | Time elapsed: 01h 32 m 45 s
Loss: 0.000933 | ΔLossTotal: -0.0060 | Iteration: 1362 | Time elapsed: 01h 32 m 49 s
Loss: 0.000975 | ΔLossTotal: -0.0059 | Iteration: 1363 | Time elapsed: 01h 32 m 53 s
Loss: 0.000942 | ΔLossTotal: -0.0059 | Iteration: 1364 | Time elapsed: 01h 32 m 57 s
Loss: 0.000954 | ΔLossTotal: -0.0059 | Iteration: 1365 | Time elapsed: 01h 33 m 01 s
Loss: 0.000875 | ΔLossTotal: -0.0060 | Iteration: 1366 | Time elapsed: 01h 33 m 05 s
Loss: 0.000904 | ΔLossTotal: -0.0060 | Iteration: 1367 | Time elapsed: 01h 33 m 09 s
Loss: 0.000870 | ΔLossTotal: -0.0060 | Iteration: 1368 | Time elapsed: 01h 33 m 13 s
Loss: 0.000905 | ΔLossTotal: -0.0060 | Iteration: 1369 | Time elapsed: 01h 33 m 17 s
Loss: 0.000842 | ΔLossTotal: -0.0060 | Iteration: 1370 | Time ela

Loss: 0.000993 | ΔLossTotal: -0.0059 | Iteration: 1456 | Time elapsed: 01h 39 m 10 s
Loss: 0.001009 | ΔLossTotal: -0.0059 | Iteration: 1457 | Time elapsed: 01h 39 m 14 s
Loss: 0.000972 | ΔLossTotal: -0.0059 | Iteration: 1458 | Time elapsed: 01h 39 m 19 s
Loss: 0.000970 | ΔLossTotal: -0.0059 | Iteration: 1459 | Time elapsed: 01h 39 m 23 s
Loss: 0.001008 | ΔLossTotal: -0.0059 | Iteration: 1460 | Time elapsed: 01h 39 m 27 s
Loss: 0.000964 | ΔLossTotal: -0.0059 | Iteration: 1461 | Time elapsed: 01h 39 m 31 s
Loss: 0.001009 | ΔLossTotal: -0.0059 | Iteration: 1462 | Time elapsed: 01h 39 m 35 s
Loss: 0.001019 | ΔLossTotal: -0.0059 | Iteration: 1463 | Time elapsed: 01h 39 m 39 s
Loss: 0.001002 | ΔLossTotal: -0.0059 | Iteration: 1464 | Time elapsed: 01h 39 m 43 s
Loss: 0.000903 | ΔLossTotal: -0.0060 | Iteration: 1465 | Time elapsed: 01h 39 m 47 s
Loss: 0.000962 | ΔLossTotal: -0.0059 | Iteration: 1466 | Time elapsed: 01h 39 m 51 s
Loss: 0.001066 | ΔLossTotal: -0.0058 | Iteration: 1467 | Time ela

Loss: 0.001087 | ΔLossTotal: -0.0058 | Iteration: 1553 | Time elapsed: 01h 45 m 45 s
Loss: 0.000963 | ΔLossTotal: -0.0059 | Iteration: 1554 | Time elapsed: 01h 45 m 49 s
Loss: 0.000947 | ΔLossTotal: -0.0059 | Iteration: 1555 | Time elapsed: 01h 45 m 53 s
Loss: 0.000910 | ΔLossTotal: -0.0060 | Iteration: 1556 | Time elapsed: 01h 45 m 57 s
Loss: 0.000988 | ΔLossTotal: -0.0059 | Iteration: 1557 | Time elapsed: 01h 46 m 01 s
Loss: 0.000946 | ΔLossTotal: -0.0059 | Iteration: 1558 | Time elapsed: 01h 46 m 05 s
Loss: 0.001018 | ΔLossTotal: -0.0059 | Iteration: 1559 | Time elapsed: 01h 46 m 09 s
Loss: 0.001022 | ΔLossTotal: -0.0059 | Iteration: 1560 | Time elapsed: 01h 46 m 13 s
Loss: 0.000962 | ΔLossTotal: -0.0059 | Iteration: 1561 | Time elapsed: 01h 46 m 17 s
Loss: 0.001048 | ΔLossTotal: -0.0058 | Iteration: 1562 | Time elapsed: 01h 46 m 21 s
Loss: 0.000984 | ΔLossTotal: -0.0059 | Iteration: 1563 | Time elapsed: 01h 46 m 25 s
Loss: 0.000951 | ΔLossTotal: -0.0059 | Iteration: 1564 | Time ela

Loss: 0.001142 | ΔLossTotal: -0.0057 | Iteration: 1650 | Time elapsed: 01h 52 m 18 s
Loss: 0.001129 | ΔLossTotal: -0.0058 | Iteration: 1651 | Time elapsed: 01h 52 m 22 s
Loss: 0.001146 | ΔLossTotal: -0.0057 | Iteration: 1652 | Time elapsed: 01h 52 m 26 s
Loss: 0.001123 | ΔLossTotal: -0.0058 | Iteration: 1653 | Time elapsed: 01h 52 m 31 s
Loss: 0.001057 | ΔLossTotal: -0.0058 | Iteration: 1654 | Time elapsed: 01h 52 m 35 s
Loss: 0.001065 | ΔLossTotal: -0.0058 | Iteration: 1655 | Time elapsed: 01h 52 m 39 s
Loss: 0.001115 | ΔLossTotal: -0.0058 | Iteration: 1656 | Time elapsed: 01h 52 m 43 s
Loss: 0.001087 | ΔLossTotal: -0.0058 | Iteration: 1657 | Time elapsed: 01h 52 m 47 s
Loss: 0.001164 | ΔLossTotal: -0.0057 | Iteration: 1658 | Time elapsed: 01h 52 m 51 s
Loss: 0.001136 | ΔLossTotal: -0.0058 | Iteration: 1659 | Time elapsed: 01h 52 m 55 s
Loss: 0.001106 | ΔLossTotal: -0.0058 | Iteration: 1660 | Time elapsed: 01h 52 m 59 s
Loss: 0.001159 | ΔLossTotal: -0.0057 | Iteration: 1661 | Time ela

In [11]:
torch.save(model.state_dict(), "network1.pth")